In [ ]:
import pandas as pd
import numpy as np
import PyUber

In [ ]:
sql = r"""SELECT 
          a0.lot7 AS lot7
         ,a0.devrevstep AS devrevstep
         ,(SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) AS last_pass
         ,a0.rework_latest_flag AS rework_latest_flag
         ,a0.lot_rework_flag AS rework_flag
         ,a1.entity AS entity
         ,a1.ceid AS ceid
         ,a2.monitor_set_name AS monitor_set_name
         ,To_Char(a2.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS data_collect_date
         ,a2.area AS area
         ,a2.monitor_process AS monitor_process
         ,a2.status AS mon_set_status
         ,a2.spc_data_id AS spc_data_id
         ,a2.violation_flag AS violation_flag
         ,a2.latest_flag AS mon_set_latest_flag
         ,a5.chart_point_seq AS chart_point_seq
         ,a5.value AS chart_value
         ,a5.process_chamber AS process_chamber
         ,a5.wafer AS chart_wafer
         ,a5.latest_flag AS chart_pt_latest_flag
         ,a5.status AS chart_pt_status
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a10.lo_control_lmt AS lo_control_lmt
         ,a10.target AS target
         ,a10.up_control_lmt AS up_control_lmt
         ,a2.test_name AS test_name
         ,a0.operation AS spc_operation
         ,a5.wafer3 AS chart_wafer3
         ,a2.monitor_type AS monitor_type
         ,a3.parameter_class AS parameter_class
         ,a9.chart_parameter AS chart_parameter
         ,a9.chart_on AS chart_on
         ,a3.measurement_set_name AS measurement_set_name
         ,a11.
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
LEFT JOIN P_SPC_CHART a9 ON a9.chart_id = a5.chart_id
LEFT JOIN P_SPC_CHART_LIMIT a10 ON a10.chart_id = a5.chart_id AND a10.limit_id = a5.limit_id
LEFT JOIN P_SPC_CHARTPOINT_MEASUREMENT a11 ON a11.sps_it == a11.sps_id and a11.measurement_set_name == a4.measurement_set_name
WHERE
              a0.lot7 = 'L210060' 
 AND      (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a0.rework_latest_flag = 'Y' 
 AND      a3.valid_flag <> 'I' 
 AND      a5.latest_flag = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.value is not NULL"""

In [ ]:
sql2 = """
SELECT 
          a0.lot7 AS lot7
         ,a0.lot_type AS lot_type
         ,a0.lot_virtual_line AS lot_virtual_line
         ,a0.operation AS spc_operation
         ,a0.devrevstep AS devrevstep
         ,To_Char(a0.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS lot_data_collect_date
         ,a1.entity AS entity
         ,To_Char(a1.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS entity_data_collect_date
         ,a1.entity_sequence AS entity_sequence
         ,a2.monitor_set_name AS monitor_set_name
         ,a3.measurement_set_name AS measurement_set_name
         ,a2.test_name AS test_name
         ,a2.spc_data_id AS spc_data_id
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a5.value AS chart_value
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
WHERE
              (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.latest_flag = 'Y' 
 AND      a0.data_collection_time >= (SYSDATE - 5/24) 
 AND      a0.lot Like 'L%' """

In [ ]:
con = PyUber.connect("F32_PROD_XEUS")

In [ ]:
df = pd.read_sql(sql, con)

In [ ]:
df = df[df['MONITOR_TYPE'] == "WIP MONITOR"]


In [ ]:
df['CHART_WAFER3'].value_counts()/df.shape[0]

In [ ]:
dfnl = df[df['CHART_WAFER3'].isna()]

In [ ]:
dfnl = dfnl[dfnl['CHART_PT_STATUS'] == 'V']

In [ ]:
dfnl['colnames'] = dfnl['SPC_OPERATION'] + '`' + dfnl['TEST_NAME'] + '`' + dfnl['MONITOR_SET_NAME'] + '`' + dfnl['MEASUREMENT_SET_NAME'] + '`' + dfnl['SPC_CHART_CATEGORY'] + '`' + dfnl['SPC_CHART_SUBSET'] + '`' + dfnl['CHART_TYPE'] + '`' + dfnl['PARAMETER_CLASS'] + '`' + dfnl['CHART_ON']
#dfnl.pivot(index=['LOT7'], columns='colnames', values = 'CHART_VALUE')

In [ ]:
sub = dfnl.groupby(['LOT7', 'colnames']).size()

In [ ]:
r = sub[sub > 1].reset_index()
#sub[sub['count']>1].head()

In [ ]:
dup = dfnl[dfnl['colnames'].isin(r['colnames'].unique())]

In [ ]:
dup['CHART_VALUE']

In [ ]:
dup['colnames'].unique().size, dfnl['colnames'].unique().size

In [ ]:
m = dup.iloc[1] != dup.iloc[0]
print(m)

In [ ]:
dup.iloc[1][m], dup.iloc[0][m]

In [ ]:
df = pd.read_sql("select * from p_spc_session LIMIT 10", con)

In [ ]:
df.head()

In [ ]:
sql = f"""
SELECT 
          a0.lot7 AS lot7
         ,a0.devrevstep AS devrevstep
         ,(SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) AS last_pass
         ,a0.rework_latest_flag AS rework_latest_flag
         ,a0.lot_rework_flag AS rework_flag
         ,a1.entity AS entity
         ,a1.ceid AS ceid
         ,a2.monitor_set_name AS monitor_set_name
         ,To_Char(a2.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS data_collect_date
         ,a2.area AS area
         ,a2.monitor_process AS process
         ,a2.status AS mon_set_status
         ,a2.spc_data_id AS spc_data_id
         ,a2.violation_flag AS violation_flag
         ,a2.latest_flag AS mon_set_latest_flag
         ,a5.chart_point_seq AS chart_point_seq
         ,a5.value AS chart_value
         ,a5.process_chamber AS process_chamber
         ,a5.wafer AS chart_wafer
         ,a5.latest_flag AS chart_pt_latest_flag
         ,a5.status AS chart_pt_status
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a10.lo_control_lmt AS lo_control_lmt
         ,a10.target AS target
         ,a10.up_control_lmt AS up_control_lmt
         ,a2.test_name AS test_name
         ,a0.operation AS spc_operation
         ,a5.wafer3 AS chart_wafer3
         ,a2.monitor_type AS monitor_type
         ,a3.parameter_class AS parameter_class
         ,a9.chart_parameter AS chart_parameter
         ,a9.chart_on AS chart_on
         ,a3.measurement_set_name AS measurement_set_name
         ,a0.load_date AS load_date
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
LEFT JOIN P_SPC_CHART a9 ON a9.chart_id = a5.chart_id
LEFT JOIN P_SPC_CHART_LIMIT a10 ON a10.chart_id = a5.chart_id AND a10.limit_id = a5.limit_id

WHERE
              SUBSTR(a0.devrevstep,1, 4) IN ('8PFU', '8PJS')
AND      a0.LOAD_DATE > :START AND a0.LOAD_DATE <= :END
 AND      (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 /*AND      a0.rework_latest_flag = 'Y'*/ 
 AND      a3.valid_flag <> 'I' 
 AND      a5.latest_flag = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.value is not NULL

"""

In [ ]:
from datetime import datetime, timedelta
df = pd.read_sql(sql, con, params={'START': (datetime.now() - timedelta(hours=2*24)), 'END':datetime.now()})

In [ ]:
df.head()

In [ ]:
def clean_dep_chamber(x):
    if not isinstance(x, str):
        return x
    
    xl = x.split(";")
    xn = []
    
    for xi in xl:
        if 'PROCESS_CHAMBER' in xi:
            continue
        else:
            xn.append(xi)
    return ";".join(xn)

df['chart_param_clean'] = df['CHART_PARAMETER'].apply(clean_dep_chamber)
df['colnames'] = df['SPC_OPERATION'] + '`' + df['TEST_NAME'] + '`' + df['MONITOR_SET_NAME'] + '`' + df['MEASUREMENT_SET_NAME'] + '`' + df['chart_param_clean'] + '`' + df['PARAMETER_CLASS']


In [ ]:
names = df['colnames'][df['colnames'].str.contains("NMT")].unique()

In [ ]:
dfg = df.pivot_table(index = ['PROCESS', 'DEVREVSTEP', 'LOT7'], columns='colnames', values='CHART_VALUE').reset_index()

In [ ]:
(dfg.loc[:, names].notnull().sum()/dfg.shape[0]).sort_values()

In [ ]:
dfg.head()

In [ ]:
[SPC-UPDATE-QUERY]
#holdBack=-(2.0/24.0)

  (select 0 as "ZERO" from dual) z
  ?*LOT_EXISTS[ inner join p_spc_lot sl on ( ?*LOT[ sl.lot like substr('?*LOT', 1,7) || '%' ?*[ OR ] ] ) ]
  inner join p_spc_session s
  ?*LOT_EXISTS[ on s.spcs_id = sl.spcs_id ]
  ?*DATEFROM[ on ?*QUERYTIME > to_date(substr('?*DATEFROM',1,19), 'YYYY-MM-DD-HH24-MI-SS') ?*/BUMP[ + 1.0/60.0/24.0/60.0 ] ?*/BACKWARDS[ - ?*MULTIPLIER[ ?*MULTIPLIER * ] ?*ITERATION ?*DAY[ * 1.0 ] ?*HOUR[ * 1.0/24.0 ] ?*MINUTE[ * 1.0/24.0/60.0 ] ] ]
  ?*DATETO[ and ?*QUERYTIME <= to_date(substr('?*DATETO',1,19), 'YYYY-MM-DD-HH24-MI-SS.FF3') ]
  ?*INTERVAL[ ?*DATEFROM[
              AND ?*QUERYTIME < to_date(substr('?*DATEFROM',1,19), 'YYYY-MM-DD-HH24-MI-SS')  + ?*MULTIPLIER[ ?*MULTIPLIER * ] ?*ITERATION ?*DAY[ * 1.0 ] ?*HOUR[ * 1.0/24.0 ] ?*MINUTE[ * 1.0/24.0/60.0 ]
              and ?*QUERYTIME < CURRENT_DATE ?*HOLDBACK -- ignore last period of time
              ?*/BACKWARDS[ AND ?*QUERYTIME <= to_date(substr('?*DATEFROM',1,19), 'YYYY-MM-DD-HH24-MI-SS') ]
              ] ]

   -- ignore some things that were wrong to put in SPC...
   and s.monitor_set_name not like 'ETT%'
   and s.monitor_set_name not like 'SRAM_ETT%'
   and s.monitor_set_name not like 'SDX%TRAY%MON'

   inner join p_spc_lot bl
    on bl.spcs_id = s.spcs_id
  and bl.lot_process like '?*PROCESS'||'%'
  and (substr(bl.product,1,1) in ( '8', 'B' )
      ?*PRODUCT_EXISTS[ or ( ?*PRODUCT[ bl.product like '?*PRODUCT' ?*[ OR ] ] ) ]
      )
  and (bl.lot_type not in ( 'MW', 'SFD' )
      ?*OWNER_EXISTS[ or ( ?*OWNER[ bl.lot_type like '?*OWNER' ?*[ OR ] ] ) ]
      )
  ?*LOT_EXISTS[ and bl.lot = sl.lot ]
  ?*WCLOT_EXISTS[ and ( ?*WCLOT[ bl.lot like '?*WCLOT' ?*[ OR ] ] ) ]
  ?*WCPRODUCT_EXISTS[ and ( ?*WCPRODUCT[ bl.product like '?*WCPRODUCT' ?*[ OR ] ] ) ]

  ! allow for an operation filter for debug
  ?*OPERATION_EXISTS[ and ( ?*OPERATION[ bl.operation like '?*OPERATION' ?*[ OR ] ] ) ]


In [ ]:
#XEUS-R Query
sql = """
   select   /*+ use_nl(z s bl rs r)
               index(s, X1P_SPC_SESSION)
               index(bl, X4P_SPC_LOT)
               index(rs, XUI_SPC_MEASUREMENT_SET)
               index(r, X4P_SPC_MEASUREMENT)
            */
   substr(bl.lot,1,7)||'0' as "LOT"
  ,bl.operation
  ,bl.spcs_id as "SPCS_ID_AS_DOUBLE"

  ,bl.lot as "ATLOT"
  ,bl.MONITOR_SET_NAME
  ,bl.TEST_NAME
  ,bl.LOT_REWORK_FLAG
  ,bl.REWORK_LATEST_FLAG
  ,bl.PRODUCT
  ,bl.LOT_TYPE as "OWNER"
  ,bl.LOT_PROCESS as "PROCESS"

  ,cast(s.LAST_UPDATE_TIME as date) as "LAST_UPDATE_TIME"
  ,s.LATEST_FLAG
  ,s.STATUS
  ,s.HOLD_LOT_FLAG
  ,s.CONFIRM_HOLD_LOT_FLAG
  ,s.VIOLATION_FLAG

  ,rs.MEASUREMENT_SET_NAME

  ,rs.VALID_FLAG
  ,rs.CORRECTED_FLAG
  ,rs.STANDARD_FLAG
  ,rs.PARAMETER_CLASS
  ,rs.PARAMETER_HEADER

  ,cast(r.DATA_COLLECTION_TIME as date) as "DATA_COLLECTION_TIME"
  ,r.WAFER
  ,r.VALUE
  ,r.LIMITS_FLAG
  ,r.S_INDEXVALUE
 ,r.EXPECTED_LOT
  ,r.PARAMETER_NAME

  ,r.ATTRIBUTE_ID as "ATTRIBUTE_ID_AS_DOUBLE"
  ,r.CUSTOM_CONTEXT_GROUP_ID

  ,r.measurement_id

  ,r.WAFER_COORDINATE_X
  ,r.WAFER_COORDINATE_Y

  ,r.native_x as "NATIVE_X_AS_DOUBLE"
  ,r.native_y as "NATIVE_Y_AS_DOUBLE"

  ,r.WAFER_LOCATION

  /*,cast(?*QUERYTIME as date) as "QUERYTIME"*/
from

  /*#INCLUDE=SPC-UPDATE-QUERY*/
   p_spc_lot sl
   inner join p_spc_session s on s.spcs_id = sl.spcs_id
   inner join p_spc_lot bl on bl.spcs_id = s.spcs_id
    
   inner join p_spc_measurement_set rs on rs.spcs_id = s.spcs_id

   inner join p_spc_measurement r
    on r.spcs_id = rs.spcs_id
   and r.measurement_set_name = rs.measurement_set_name

   where s.LOAD_DATE > :START and s.LOAD_DATE < :END and substr(bl.product, 1, 4) in ('8PEH')

/*order by bl.lot, r.EXPECTED_LOT, bl.operation*/"""


In [ ]:
df = pd.read_sql(sql, con, params={'START': (datetime.now() - timedelta(days=1)), 'END':datetime.now()})

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df['colsnames'] = df['OPERATION'] + '`' + df['TEST_NAME'] + "`" + df['MONITOR_SET_NAME'] + "`" + df['MEASUREMENT_SET_NAME'] + '`' + df['PARAMETER_NAME'] + '`' + df['PARAMETER_CLASS']

In [ ]:
df1 = df.groupby(['LOT', 'WAFER', 'OPERATION', 'TEST_NAME', 'MONITOR_SET_NAME', 'MEASUREMENT_SET_NAME', 'PARAMETER_NAME', 'PARAMETER_CLASS']).agg(VALUE = pd.NamedAgg(column='VALUE', aggfunc=np.mean))

dfg = df.pivot_table(index=['LOT', 'WAFER'], columns='colsnames', values='VALUE')

In [ ]:
df1.head()

In [ ]:
df1 = df1.groupby(['LOT', 'WAFER', 'OPERATION', 'TEST_NAME', 'MONITOR_SET_NAME', 'MEASUREMENT_SET_NAME', 'PARAMETER_NAME', 'PARAMETER_CLASS'])['VALUE'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df1 = df1.reset_index()

In [ ]:
df1['colsnames'] = df1['OPERATION'] + '`' + df1['TEST_NAME'] + "`" + df1['MONITOR_SET_NAME'] + "`" + df1['MEASUREMENT_SET_NAME'] + '`' + df1['PARAMETER_NAME'] + '`' + df1['PARAMETER_CLASS']

In [ ]:
df1 = df1.pivot_table(index=['LOT', 'WAFER'], columns='colsnames', values='VALUE')

In [ ]:
(df1.notnull().sum()/df1.shape[0]).sort_values()

In [ ]:
dfg2 = dfg.groupby(by=['LOT']).transform(lambda x: x.fillna(x.mean()))

In [ ]:
(dfg.notnull().sum()/dfg.shape[0]).sort_values()

In [ ]:
(dfg2.notnull().sum()/dfg2.shape[0]).sort_values()